# Workshop 3: Data Pre-processing

COSC2671 Social Media and Network Analytics

Jeffrey Chan, RMIT University, 2023


This automatically reloads the client information if there are changes.

In [2]:
%load_ext autoreload
%autoreload 1
%aimport redditClient

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Some necessary imports.

In [3]:
from redditClient import redditClient
import praw
import json
import functools
from prawcore.exceptions import TooManyRequests
import datetime
import time

Parameters used throughout this notebook.
Usually useful to have these parameters in one cell (and one location) and can edit them as needed.

In [4]:
# subreddit name we interested in getting the hot submissions
sSubredditName = 'NVDA_Stock'
# maximum number of hot submissions
hotLimit = 5

Construct Reddit client then print our name to test.

In [5]:
# construct Reddit client
client = redditClient()

# sanity check, you should see your own username printed out
print(client.user.me())

Comprehensive-End522


In [6]:
@functools.lru_cache(maxsize=None)
def reddit_api_response(sSubredditName, hotLimit,start_date, end_date):  
    # specify which subreddit we are interested in
    subreddit = client.subreddit(sSubredditName)
    # Convert dates to timestamps
    start_timestamp = int(start_date.timestamp())
    end_timestamp = int(end_date.timestamp())
    data = []; comments_json = []        
    for submission in subreddit.top(limit=None):
        try:
            submission.comments.replace_more(limit=None) # expand all the comments
            if start_timestamp <= submission.created_utc <= end_timestamp:
                for comment in submission.comments.list():
                    comments=[]
                    comment_data={
                        "author": comment.author.name if comment.author else "No Author",
                        "created": comment.created_utc,
                        "text": comment.body
                    }
                    comments.append(comment_data)            
                post_data = {
                "title": submission.title,
                "author": submission.author.name if submission.author else "No Author",
                "score": submission.score,
                "created":submission.created_utc
                }
                data.append(post_data)
            # else:
            #     break
        except TooManyRequests as e:
            time.sleep(60)
    print(data)
    final_data = {"submissions":data}
    with open("nvidia_stock_data.json", "w") as json_file:
        json.dump(final_data, json_file)
    print("Data saved to nvidia_stock_data.json")
    with open("nvidia_stock_data_comments.json", "w") as json_file:
        json.dump(comments_json, json_file)
    print("Comments Data saved to nvidia_stock_data_comments.json")

In [7]:
start_date = datetime.datetime(2024, 1, 1, 0, 0, 0)  # Year, Month, Day, Hour, Minute, Second
end_date = datetime.datetime(2024, 8, 15, 0, 5, 0)
reddit_api_response(sSubredditName, hotLimit,start_date, end_date)

[{'title': 'Funny story about how NVDA made me a multi millionaire.', 'author': 'Sleepyj7791', 'score': 2297, 'created': 1719014335.0}, {'title': 'How it feels holding NVDA right now.', 'author': 'CaptainSebz', 'score': 1214, 'created': 1722364714.0}, {'title': 'It was just at 108…', 'author': 'hshshebahjsna', 'score': 1104, 'created': 1723061283.0}, {'title': 'Jensen Huang - Annual Shareholder Meeting Notes', 'author': 'Xtianus21', 'score': 703, 'created': 1719419076.0}, {'title': 'We did it', 'author': 'sleepavenue', 'score': 644, 'created': 1716412537.0}, {'title': 'Video of Nvidia CEO entering the earnings call today ', 'author': 'Responsible_Hotel_65', 'score': 634, 'created': 1716409621.0}, {'title': "It's the same script every quarter...", 'author': 'kellyclarksn', 'score': 574, 'created': 1719251172.0}, {'title': 'Holding…', 'author': 'hshshebahjsna', 'score': 569, 'created': 1721915641.0}, {'title': 'NVDA 50% rally coming', 'author': 'Dieselcock', 'score': 565, 'created': 1722

In [8]:
print(client.auth.limits)

{'remaining': 828.0, 'reset_timestamp': 1724394000.4028265, 'used': 172}


In [9]:
print(datetime.datetime.fromtimestamp(1723608000.1579154))

2024-08-14 04:00:00.157915
